In [1]:
from langchain.agents import Tool
from langchain.chains import RetrievalQA
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from pydantic import BaseModel, Field

In [2]:
import os
os.environ["OPENAI_API_KEY"] = "your key"
os.environ["GOOGLE_CSE_ID"] = "your id"
os.environ["GOOGLE_API_KEY"] = "your key"

In [3]:
from langchain.document_loaders import ArxivLoader

## Arxivにおける論文の検索とデータの抽出

In [4]:
class DocumentInput(BaseModel):
    question: str = Field()

llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613")

name1 = "Chain-of-Thought Prompting Elicits Reasoning in Large Language Models"
no1 = "2201.11903"
name2 = "ReAct: Synergizing Reasoning and Acting in Language Models"
no2 = "2210.03629"

tools = []
files = [
    {
        "name": name1[:5],
        "no": no1,
    },
    {
        "name": name2[:5],
        "no": no2,
    },
]

for file in files:
    pages = ArxivLoader(file["no"], load_max_docs=2).load_and_split()
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    docs = text_splitter.split_documents(pages)
    embeddings = OpenAIEmbeddings()
    retriever = FAISS.from_documents(docs, embeddings).as_retriever()

    tools.append(
        Tool(
            args_schema=DocumentInput,
            name=file["name"],
            description=f"useful when you want to answer questions about {file['name']}",
            func=RetrievalQA.from_chain_type(llm=llm, retriever=retriever),
        )
    )

/Users/nagayoshi3/.pyenv/versions/anaconda3-2020.07/envs/langchain_book_master/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
/Users/nagayoshi3/.pyenv/versions/anaconda3-2020.07/envs/langchain_book_master/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai

## LLMによる検索

In [5]:
from langchain.agents import AgentType, initialize_agent

In [6]:
llm = ChatOpenAI(
    temperature=0,
    model="gpt-3.5-turbo-0613",
#    model="gpt-4-1106-preview",
)

agent = initialize_agent(
    agent=AgentType.OPENAI_FUNCTIONS,
    tools=tools,
    llm=llm,
    verbose=True,
)

agent({"input": f"{name1}と{name2}の共通点について教えてください。"})
agent({"input": f"{name1}と{name2}の違いについて教えてください。"})

/Users/nagayoshi3/.pyenv/versions/anaconda3-2020.07/envs/langchain_book_master/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(
/Users/nagayoshi3/.pyenv/versions/anaconda3-2020.07/envs/langchain_book_master/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
「Chain-of-Thought Prompting Elicits Reasoning in Large Language Models」と「ReAct: Synergizing Reasoning and Acting in Language Models」は、両方とも大規模な言語モデルにおける推論能力の向上を目指した研究です。具体的な共通点は以下の通りです：

1. 目的：両研究は、言語モデルの推論能力を向上させることを目的としています。これにより、モデルがより複雑なタスクや問題に対してより洞察力のある回答を生成できるようになります。

2. プロンプティング：両研究では、プロンプティングと呼ばれる手法が使用されています。プロンプティングは、モデルに対して特定の情報や質問を与えることで、推論や応答の品質を向上させるための手法です。

3. 推論の促進：両研究では、推論を促進するための手法が提案されています。具体的には、Chain-of-Thought Promptingでは、モデルに対して連鎖的な質問を提示することで、推論の連続性を促します。一方、ReActでは、モデルに対して行動を促すための情報を与えることで、推論と行動の統合を図ります。

4. モデルの改善：両研究では、提案された手法によって言語モデルの性能が向上することが示されています。具体的な改善点は研究ごとに異なりますが、いずれの手法も推論能力の向上に成功しています。

以上が、「Chain-of-Thought Prompting Elicits Reasoning in Large Language Models」と「ReAct: Synergizing Reasoning and Acting in Language Models」の共通点です。

> Finished chain.


> Entering new AgentExecutor chain...
「Chain-of-Thought Prompting Elicits Reasoning in Large Language Models」と「ReAct: Synergizing Reasoning and Acting in Language Mo

{'input': 'Chain-of-Thought Prompting Elicits Reasoning in Large Language ModelsとReAct: Synergizing Reasoning and Acting in Language Modelsの違いについて教えてください。',
 'output': '「Chain-of-Thought Prompting Elicits Reasoning in Large Language Models」と「ReAct: Synergizing Reasoning and Acting in Language Models」は、両方とも言語モデルの推論能力を向上させるための手法ですが、アプローチや目的にはいくつかの違いがあります。\n\n「Chain-of-Thought Prompting Elicits Reasoning in Large Language Models」は、言語モデルが推論を行う際に、連鎖的な思考を促すプロンプトを使用する手法です。具体的には、モデルに対して複数の質問を連続して提示し、前の質問の回答を次の質問の文脈として活用することで、より複雑な推論を行わせることができます。この手法は、モデルが長期的な文脈を理解し、推論を進める能力を向上させることを目指しています。\n\n一方、「ReAct: Synergizing Reasoning and Acting in Language Models」は、言語モデルが推論だけでなく、行動も行えるようにする手法です。具体的には、モデルに対して行動を指示するプロンプトを与えることで、モデルが推論結果を実際の行動に反映させることができます。この手法は、モデルが推論結果を実際のタスクに応用し、より実用的な応答を生成する能力を向上させることを目指しています。\n\nつまり、「Chain-of-Thought Prompting Elicits Reasoning in Large Language Models」は、推論の複雑さを増すために連鎖的な思考を促す手法であり、「ReAct: Synergizing Reasoning and Acting in Language Models」は、推論結果を実際の行動に反映させるための手法です